In [6]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from generic_db import GenericDBOperations
from tqdm import tqdm
import re

In [2]:
from transformers import pipeline

In [3]:
pd.set_option('display.expand_frame_repr', False)
# pd.set_option('display.max_colwidth', None)

In [4]:
# generic_db = GenericDBOperations()

In [4]:
doc_info = pd.read_csv('doc_info.csv', index_col=0)
doc_info

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document,sub,avg_score,doc_count
0,Why air conditioners can be a problematic sol...,2,2_conditioners_people_health_government,"['conditioners', 'people', 'health', 'governme...",['Highlights: * The province was supposed to c...,conditioners - people - health - government - ...,1.000000,False,canada,29.285714,7
1,Bit of a alarmist clickbait headline. I agree ...,-1,-1_like_people_building_window,"['like', 'people', 'building', 'window', 'day'...","['In the meantime, turn on the oven at 450F an...",like - people - building - window - day - just...,0.000000,False,canada,10.000000,2
2,"I just lived thru a week of 45°c. No, Im not t...",2,2_conditioners_people_health_government,"['conditioners', 'people', 'health', 'governme...",['Highlights: * The province was supposed to c...,conditioners - people - health - government - ...,0.889142,False,canada,7.222222,9
3,Serious question for anyone who cares to answe...,2,2_conditioners_people_health_government,"['conditioners', 'people', 'health', 'governme...",['Highlights: * The province was supposed to c...,conditioners - people - health - government - ...,0.745981,False,canada,1.000000,3
4,Grasshoppers threaten to devour Alberta crops...,-1,-1_like_people_building_window,"['like', 'people', 'building', 'window', 'day'...","['In the meantime, turn on the oven at 450F an...",like - people - building - window - day - just...,0.000000,False,canada,26.444444,18
...,...,...,...,...,...,...,...,...,...,...,...
1347,"Choosing to believe the weather network, I’m ...",1,1_weather_climate_rain_records,"['weather', 'climate', 'rain', 'records', 'wav...",['However its been the worst fire season ever ...,weather - climate - rain - records - wave - ch...,1.000000,False,VictoriaBC,4.857143,7
1348,Every Monday at six Classic cars and Hot Rods...,1,1_weather_climate_rain_records,"['weather', 'climate', 'rain', 'records', 'wav...",['However its been the worst fire season ever ...,weather - climate - rain - records - wave - ch...,1.000000,False,VictoriaBC,4.500000,2
1349,Beer for long hot summer days Do any of the l...,-1,-1_like_people_building_window,"['like', 'people', 'building', 'window', 'day'...","['In the meantime, turn on the oven at 450F an...",like - people - building - window - day - just...,0.000000,False,VictoriaBC,16.545455,11
1350,Am I the only one who thinks we should have mo...,-1,-1_like_people_building_window,"['like', 'people', 'building', 'window', 'day'...","['In the meantime, turn on the oven at 450F an...",like - people - building - window - day - just...,0.000000,False,VictoriaBC,1.333333,18


In [7]:
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english", truncation=True)

In [8]:
sentiment_analysis(doc_info['Document'].to_list()[0])

[{'label': 'NEGATIVE', 'score': 0.9949689507484436}]

In [9]:
sentiment_direction = []
sentiment_score = []

for doc in tqdm(doc_info['Document'].to_list()):
    sa = sentiment_analysis(doc)
    sentiment_direction.append(sa[0]['label'])
    sentiment_score.append(sa[0]['score'])

100%|██████████████████████████████████████████████████████████████████████████████| 1352/1352 [37:08<00:00,  1.65s/it]


In [10]:
doc_info['sentiment_direction'] = sentiment_direction
doc_info['sentiment_score'] = sentiment_score
doc_info

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document,sub,avg_score,doc_count,sentiment_direction,sentiment_score
0,Why air conditioners can be a problematic sol...,2,2_conditioners_people_health_government,"['conditioners', 'people', 'health', 'governme...",['Highlights: * The province was supposed to c...,conditioners - people - health - government - ...,1.000000,False,canada,29.285714,7,NEGATIVE,0.994969
1,Bit of a alarmist clickbait headline. I agree ...,-1,-1_like_people_building_window,"['like', 'people', 'building', 'window', 'day'...","['In the meantime, turn on the oven at 450F an...",like - people - building - window - day - just...,0.000000,False,canada,10.000000,2,POSITIVE,0.994842
2,"I just lived thru a week of 45°c. No, Im not t...",2,2_conditioners_people_health_government,"['conditioners', 'people', 'health', 'governme...",['Highlights: * The province was supposed to c...,conditioners - people - health - government - ...,0.889142,False,canada,7.222222,9,NEGATIVE,0.998972
3,Serious question for anyone who cares to answe...,2,2_conditioners_people_health_government,"['conditioners', 'people', 'health', 'governme...",['Highlights: * The province was supposed to c...,conditioners - people - health - government - ...,0.745981,False,canada,1.000000,3,NEGATIVE,0.994940
4,Grasshoppers threaten to devour Alberta crops...,-1,-1_like_people_building_window,"['like', 'people', 'building', 'window', 'day'...","['In the meantime, turn on the oven at 450F an...",like - people - building - window - day - just...,0.000000,False,canada,26.444444,18,NEGATIVE,0.998752
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1347,"Choosing to believe the weather network, I’m ...",1,1_weather_climate_rain_records,"['weather', 'climate', 'rain', 'records', 'wav...",['However its been the worst fire season ever ...,weather - climate - rain - records - wave - ch...,1.000000,False,VictoriaBC,4.857143,7,POSITIVE,0.997789
1348,Every Monday at six Classic cars and Hot Rods...,1,1_weather_climate_rain_records,"['weather', 'climate', 'rain', 'records', 'wav...",['However its been the worst fire season ever ...,weather - climate - rain - records - wave - ch...,1.000000,False,VictoriaBC,4.500000,2,POSITIVE,0.998881
1349,Beer for long hot summer days Do any of the l...,-1,-1_like_people_building_window,"['like', 'people', 'building', 'window', 'day'...","['In the meantime, turn on the oven at 450F an...",like - people - building - window - day - just...,0.000000,False,VictoriaBC,16.545455,11,POSITIVE,0.998412
1350,Am I the only one who thinks we should have mo...,-1,-1_like_people_building_window,"['like', 'people', 'building', 'window', 'day'...","['In the meantime, turn on the oven at 450F an...",like - people - building - window - day - just...,0.000000,False,VictoriaBC,1.333333,18,POSITIVE,0.998698


In [11]:
doc_info.to_csv('doc_info_with_sentiment.csv')

In [12]:
emotion_analysis = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

In [13]:
emotion_analysis(doc_info['Document'].to_list()[0])

[[{'label': 'neutral', 'score': 0.4596382677555084},
  {'label': 'curiosity', 'score': 0.1499285250902176},
  {'label': 'disapproval', 'score': 0.1406150460243225},
  {'label': 'approval', 'score': 0.10219928622245789},
  {'label': 'confusion', 'score': 0.09089447557926178},
  {'label': 'annoyance', 'score': 0.08414309471845627},
  {'label': 'disappointment', 'score': 0.021590374410152435},
  {'label': 'realization', 'score': 0.015230780467391014},
  {'label': 'admiration', 'score': 0.010227023623883724},
  {'label': 'optimism', 'score': 0.009361425414681435},
  {'label': 'disgust', 'score': 0.006337975617498159},
  {'label': 'anger', 'score': 0.0059382114559412},
  {'label': 'desire', 'score': 0.0026099663227796555},
  {'label': 'caring', 'score': 0.00258543505333364},
  {'label': 'surprise', 'score': 0.0025464093778282404},
  {'label': 'sadness', 'score': 0.001725304638966918},
  {'label': 'embarrassment', 'score': 0.001420947490260005},
  {'label': 'excitement', 'score': 0.001378020

In [14]:
all_labels = [i['label'] for i in emotion_analysis(doc_info['Document'].to_list()[0])[0]]
all_labels = {i: [] for i in all_labels}
all_labels

{'neutral': [],
 'curiosity': [],
 'disapproval': [],
 'approval': [],
 'confusion': [],
 'annoyance': [],
 'disappointment': [],
 'realization': [],
 'admiration': [],
 'optimism': [],
 'disgust': [],
 'anger': [],
 'desire': [],
 'caring': [],
 'surprise': [],
 'sadness': [],
 'embarrassment': [],
 'excitement': [],
 'fear': [],
 'amusement': [],
 'nervousness': [],
 'love': [],
 'joy': [],
 'relief': [],
 'grief': [],
 'remorse': [],
 'gratitude': [],
 'pride': []}

In [15]:
for doc in tqdm(doc_info['Document'].to_list()):
    ea = emotion_analysis(doc, padding=True, truncation=True)[0]
    for emotion in ea:
        all_labels[emotion['label']].append(emotion['score'])

100%|██████████████████████████████████████████████████████████████████████████████| 1352/1352 [11:11<00:00,  2.01it/s]


In [16]:
for l, s in all_labels.items():
    doc_info[l] = s

doc_info

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document,sub,avg_score,...,fear,amusement,nervousness,love,joy,relief,grief,remorse,gratitude,pride
0,Why air conditioners can be a problematic sol...,2,2_conditioners_people_health_government,"['conditioners', 'people', 'health', 'governme...",['Highlights: * The province was supposed to c...,conditioners - people - health - government - ...,1.000000,False,canada,29.285714,...,0.001341,0.001125,0.000757,0.000625,0.000559,0.000441,0.000398,0.000347,0.000262,0.000254
1,Bit of a alarmist clickbait headline. I agree ...,-1,-1_like_people_building_window,"['like', 'people', 'building', 'window', 'day'...","['In the meantime, turn on the oven at 450F an...",like - people - building - window - day - just...,0.000000,False,canada,10.000000,...,0.001133,0.006358,0.000799,0.000733,0.002315,0.001826,0.000347,0.000339,0.000588,0.001201
2,"I just lived thru a week of 45°c. No, Im not t...",2,2_conditioners_people_health_government,"['conditioners', 'people', 'health', 'governme...",['Highlights: * The province was supposed to c...,conditioners - people - health - government - ...,0.889142,False,canada,7.222222,...,0.000682,0.001224,0.000609,0.000347,0.000554,0.000353,0.000289,0.000317,0.000240,0.000232
3,Serious question for anyone who cares to answe...,2,2_conditioners_people_health_government,"['conditioners', 'people', 'health', 'governme...",['Highlights: * The province was supposed to c...,conditioners - people - health - government - ...,0.745981,False,canada,1.000000,...,0.000909,0.001141,0.000519,0.000800,0.000690,0.000230,0.000252,0.000540,0.000485,0.000106
4,Grasshoppers threaten to devour Alberta crops...,-1,-1_like_people_building_window,"['like', 'people', 'building', 'window', 'day'...","['In the meantime, turn on the oven at 450F an...",like - people - building - window - day - just...,0.000000,False,canada,26.444444,...,0.005539,0.000989,0.002166,0.000950,0.001361,0.000881,0.000592,0.000979,0.000243,0.000709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1347,"Choosing to believe the weather network, I’m ...",1,1_weather_climate_rain_records,"['weather', 'climate', 'rain', 'records', 'wav...",['However its been the worst fire season ever ...,weather - climate - rain - records - wave - ch...,1.000000,False,VictoriaBC,4.857143,...,0.000965,0.002616,0.001063,0.528251,0.047634,0.001114,0.000376,0.000468,0.001491,0.000878
1348,Every Monday at six Classic cars and Hot Rods...,1,1_weather_climate_rain_records,"['weather', 'climate', 'rain', 'records', 'wav...",['However its been the worst fire season ever ...,weather - climate - rain - records - wave - ch...,1.000000,False,VictoriaBC,4.500000,...,0.000508,0.004015,0.000311,0.001766,0.018861,0.003643,0.000326,0.000635,0.975323,0.000836
1349,Beer for long hot summer days Do any of the l...,-1,-1_like_people_building_window,"['like', 'people', 'building', 'window', 'day'...","['In the meantime, turn on the oven at 450F an...",like - people - building - window - day - just...,0.000000,False,VictoriaBC,16.545455,...,0.000448,0.001101,0.000415,0.018005,0.004175,0.000882,0.000200,0.000421,0.000595,0.000526
1350,Am I the only one who thinks we should have mo...,-1,-1_like_people_building_window,"['like', 'people', 'building', 'window', 'day'...","['In the meantime, turn on the oven at 450F an...",like - people - building - window - day - just...,0.000000,False,VictoriaBC,1.333333,...,0.000466,0.003020,0.000886,0.047547,0.093537,0.008483,0.000491,0.000750,0.052463,0.004947


In [17]:
doc_info.to_csv('doc_info_with_sentiment_and_emotion.csv')